# Phase V: CNN/LSTM Model Training

This notebook trains the CNN/LSTM gesture recognition model using continuous motion data.

## Training Pipeline:
1. Load continuous sensor recordings
2. Load label files
3. Create sliding window dataset
4. Split train/validation/test sets
5. Train model with early stopping
6. Evaluate performance
7. Save trained model

Expected training time: 1-2 hours on CPU, 10-20 minutes on GPU

In [ ]:
# Import required libraries
import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Import our model
from models.cnn_lstm_model import (
    create_cnn_lstm_model,
    prepare_data_for_training,
    save_model
)

print("✓ Imports successful")

## Step 1: Load Data

Load continuous sensor recordings and their corresponding labels.

In [ ]:
# Configure data paths
DATA_DIR = '../data/continuous'
SESSION_FILES = [
    # Add your session files here
    # ('session_01.csv', 'session_01_labels.csv'),
    # ('session_02.csv', 'session_02_labels.csv'),
]

# For demo: use a placeholder
if not SESSION_FILES:
    print("⚠️  No session files configured.")
    print("   Add your continuous recording files to SESSION_FILES list above.")
    print("   Example: ('session_01.csv', 'session_01_labels.csv')")
else:
    print(f"Configured {len(SESSION_FILES)} session(s) for training")

In [ ]:
# Load and prepare data from all sessions
X_all = []
y_all = []

for sensor_file, labels_file in SESSION_FILES:
    print(f"\nProcessing {sensor_file}...")
    
    # Load files
    sensor_path = f"{DATA_DIR}/{sensor_file}"
    labels_path = f"{DATA_DIR}/{labels_file}"
    
    sensor_data = pd.read_csv(sensor_path)
    labels_data = pd.read_csv(labels_path)
    
    print(f"  Sensor data: {len(sensor_data)} samples")
    print(f"  Labels: {len(labels_data)} segments")
    
    # Create sliding windows
    X, y = prepare_data_for_training(
        sensor_data,
        labels_data,
        window_size=50,  # 1 second at 50Hz
        stride=25        # 50% overlap
    )
    
    print(f"  Generated {len(X)} training windows")
    
    X_all.append(X)
    y_all.append(y)

# Combine all sessions
if X_all:
    X_combined = np.concatenate(X_all, axis=0)
    y_combined = np.concatenate(y_all, axis=0)
    
    print(f"\n✓ Total training samples: {len(X_combined)}")
    print(f"  Input shape: {X_combined.shape}")
    print(f"  Output shape: {y_combined.shape}")
else:
    print("\n⚠️  No data loaded. Configure SESSION_FILES above.")

## Step 2: Data Analysis

Analyze class distribution and data quality.

In [ ]:
if 'y_combined' in locals():
    # Gesture names
    gesture_names = ['Jump', 'Punch', 'Turn', 'Walk', 'Noise']
    
    # Count samples per class
    class_counts = y_combined.sum(axis=0).astype(int)
    
    # Display distribution
    print("Class Distribution:")
    for i, name in enumerate(gesture_names):
        count = class_counts[i]
        pct = (count / len(y_combined)) * 100
        print(f"  {name:8s}: {count:5d} samples ({pct:5.1f}%)")
    
    # Visualize
    plt.figure(figsize=(10, 6))
    plt.bar(gesture_names, class_counts)
    plt.title('Gesture Class Distribution')
    plt.ylabel('Number of Samples')
    plt.xlabel('Gesture')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Check for imbalance
    min_count = class_counts.min()
    max_count = class_counts.max()
    imbalance_ratio = max_count / min_count
    
    if imbalance_ratio > 3:
        print(f"\n⚠️  Class imbalance detected (ratio: {imbalance_ratio:.1f}:1)")
        print("   Consider collecting more data for underrepresented classes.")
    else:
        print(f"\n✓ Class distribution is reasonably balanced (ratio: {imbalance_ratio:.1f}:1)")

## Step 3: Split Data

Split into training, validation, and test sets.

In [ ]:
if 'X_combined' in locals():
    # Split: 70% train, 15% validation, 15% test
    X_temp, X_test, y_temp, y_test = train_test_split(
        X_combined, y_combined,
        test_size=0.15,
        random_state=42,
        stratify=np.argmax(y_combined, axis=1)
    )
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp,
        test_size=0.176,  # 0.176 of 85% ≈ 15% of total
        random_state=42,
        stratify=np.argmax(y_temp, axis=1)
    )
    
    print(f"Training set:   {len(X_train)} samples ({len(X_train)/len(X_combined)*100:.1f}%)")
    print(f"Validation set: {len(X_val)} samples ({len(X_val)/len(X_combined)*100:.1f}%)")
    print(f"Test set:       {len(X_test)} samples ({len(X_test)/len(X_combined)*100:.1f}%)")
    print(f"\n✓ Data split complete")

## Step 4: Create Model

Instantiate the CNN/LSTM architecture.

In [ ]:
# Create model
model = create_cnn_lstm_model(
    input_shape=(50, 9),  # 50 timesteps, 9 features
    num_classes=5,        # 5 gestures
    cnn_filters=(64, 128),
    lstm_units=(128, 64),
    dense_units=64,
    dropout_rate=0.3
)

print("✓ Model created")
model.summary()

## Step 5: Train Model

Train with early stopping to prevent overfitting.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Configure callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    ),
    ModelCheckpoint(
        '../models/cnn_lstm_best.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# Train model
if 'X_train' in locals():
    print("Starting training...\n")
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=32,
        callbacks=callbacks,
        verbose=1
    )
    print("\n✓ Training complete")
else:
    print("⚠️  No training data available")

## Step 6: Visualize Training

Plot training and validation metrics.

In [ ]:
if 'history' in locals():
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot accuracy
    ax1.plot(history.history['accuracy'], label='Train')
    ax1.plot(history.history['val_accuracy'], label='Validation')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)
    
    # Plot loss
    ax2.plot(history.history['loss'], label='Train')
    ax2.plot(history.history['val_loss'], label='Validation')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Print final metrics
    print(f"\nFinal Training Accuracy: {history.history['accuracy'][-1]:.4f}")
    print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")

## Step 7: Evaluate on Test Set

Final evaluation on held-out test data.

In [ ]:
if 'X_test' in locals():
    # Evaluate
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
    
    # Predictions
    y_pred = model.predict(X_test, verbose=0)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(
        y_true_classes,
        y_pred_classes,
        target_names=gesture_names
    ))
    
    # Confusion matrix
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=gesture_names,
        yticklabels=gesture_names
    )
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

## Step 8: Save Model

Save the trained model for deployment.

In [ ]:
# Save final model
model_path = '../models/cnn_lstm_gesture.h5'
save_model(model, model_path)

print(f"\n✓ Model saved to {model_path}")
print("\nReady for real-time integration!")
print("Next step: Use this model in src/udp_listener_v3.py")

## Summary

Training complete! The model is now ready for real-time gesture recognition.

**Next Steps:**
1. Test the model with `src/udp_listener_v3.py`
2. Collect more data if accuracy is insufficient
3. Tune hyperparameters if needed
4. Deploy to production

**Expected Performance:**
- Accuracy: 90-98%
- Latency: 10-30ms per prediction
- Model size: ~850 KB